In [1]:
import os
import itertools
from IPython.display import display

import numpy as np
import pandas as pd
import scipy.stats as sts

from tqdm import tqdm

from typing import Callable, Tuple, Any
from functools import partial

import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.4f}'.format

In [2]:
# STATISTIC = "./matrix/new_cat_jacc_sorted.csv"
# BERT = "./matrix/scores_bert.csv"
# MEDBERT = "./matrix/scores_medbert.csv"
# DEEPSEEK = "./matrix/scores_DeepSeek-V3_mean.csv"
# MLM = "./matrix/scores_mlm.csv"
# YANDEX = "./matrix/scores_yandex_gpt5.csv"
# DOC = "./matrix/scores_yandex_doc_search.csv"
# QWEN = "./matrix/scores_qwen3-235b-a22b_0.csv"

STATISTIC = "./matrices/new_cat_jacc_sorted.csv"
BERT = "./matrices/similarity_scores.csv"
MEDBERT = "./matrix/scores_medbert.csv"
DEEPSEEK = "./matrices/scores_DeepSeek-V3_mean.csv"
MLM = "./matrices/mlm_results_final.csv"
YANDEX = "./matrices/yandex_prompt_matrix.csv"
DOC = "./matrices/scores_llm_doc.csv"
QWEN = "./matrices/scores_qwen3-235b-a22b_0.csv"

In [3]:
statistic = pd.read_csv(STATISTIC).set_index("Unnamed: 0", drop=True)
bert = pd.read_csv(BERT).set_index("Unnamed: 0", drop=True)
medbert = pd.read_csv(MEDBERT).set_index("Unnamed: 0", drop=True)
deepseek = pd.read_csv(DEEPSEEK).set_index("Unnamed: 0", drop=True)
mlm = pd.read_csv(MLM).set_index("Unnamed: 0", drop=True)
yandex = pd.read_csv(YANDEX).set_index("Unnamed: 0", drop=True)
doc = pd.read_csv(DOC).set_index("Unnamed: 0", drop=True)
qwen = pd.read_csv(QWEN).set_index("Unnamed: 0", drop=True)

In [4]:
matrices = {
    "statistic": statistic,
    "bert": bert,
    "medbert": medbert,
    "deepseek": deepseek,
    "mlm": mlm,
    "yandex": yandex,
    "doc": doc,
    "qwen": qwen
}

LLMS = ["deepseek", "qwen", "yandex", "mlm"]
BASELINES = ["statistic", "medbert", "bert", "doc"]

In [5]:
for k, v in matrices.items():
    print(k, v.shape)
    display(v.head())

statistic (1757, 1757)


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.0000,0.0400,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A02,0.0400,1.0000,0.0000,0.0035,0.0000,0.0000,0.0000,0.0043,0.0020,0.0042,...,0.0014,0.0008,0.0015,0.0010,0.0027,0.0008,0.0010,0.0016,0.0009,0.0004
A03,0.0000,0.0000,1.0000,0.0008,0.0000,0.0000,0.0133,0.0003,0.0021,0.0000,...,0.0001,0.0001,0.0002,0.0000,0.0000,0.0000,0.0001,0.0002,0.0000,0.0001
A04,0.0000,0.0035,0.0008,1.0000,0.0019,0.0002,0.0014,0.0462,0.0132,0.0025,...,0.0504,0.0433,0.0480,0.0659,0.0577,0.0392,0.0376,0.0336,0.0419,0.0634
A05,0.0000,0.0000,0.0000,0.0019,1.0000,0.0000,0.0000,0.0070,0.0041,0.0000,...,0.0007,0.0010,0.0012,0.0010,0.0024,0.0005,0.0009,0.0009,0.0006,0.0007


bert (1696, 1696)


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.0000,0.8591,0.6663,0.8590,0.8232,0.7387,0.8297,0.8829,0.8726,0.8202,...,0.8054,0.7709,0.7201,0.7007,0.7522,0.7798,0.7637,0.7390,0.7087,0.7486
A02,0.8591,1.0000,0.7781,0.9732,0.8877,0.8647,0.9421,0.9341,0.9137,0.8221,...,0.8340,0.8226,0.8044,0.8205,0.8043,0.7879,0.8504,0.8559,0.8342,0.8135
A03,0.6663,0.7781,1.0000,0.7914,0.7905,0.9191,0.8056,0.7389,0.7291,0.6688,...,0.7761,0.7331,0.7604,0.8092,0.7333,0.6571,0.7582,0.7807,0.8348,0.7784
A04,0.8590,0.9732,0.7914,1.0000,0.8980,0.8693,0.9584,0.9494,0.9096,0.8111,...,0.8440,0.8284,0.8417,0.8409,0.8171,0.7896,0.8612,0.8814,0.8637,0.8327
A05,0.8232,0.8877,0.7905,0.8980,1.0000,0.8485,0.8820,0.8853,0.8774,0.8238,...,0.8873,0.8659,0.8055,0.8212,0.8168,0.7795,0.8332,0.8581,0.8606,0.8815


medbert (1757, 1757)


...

,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.0000,0.7000,0.0000,0.0000,0.0000,0.0000,0.0000,0.6000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A02,0.7000,1.0000,0.7000,0.6000,0.0000,0.0000,0.0000,0.8000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A03,0.0000,0.7000,1.0000,0.7000,0.6000,0.7000,0.0000,0.8000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A04,0.0000,0.6000,0.7000,1.0000,0.9000,0.0000,0.7000,0.8000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A05,0.0000,0.0000,0.6000,0.9000,1.0000,0.0000,0.0000,0.8000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


doc (1696, 1696)


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.0000,0.7428,0.7808,0.7677,0.7582,0.7838,0.7795,0.7592,0.7595,0.7714,...,0.6238,0.6441,0.6317,0.5498,0.6119,0.5632,0.5865,0.6073,0.6510,0.5643
A02,0.7428,1.0000,0.7807,0.8987,0.8605,0.7332,0.8561,0.8724,0.8412,0.7605,...,0.5522,0.5543,0.5303,0.4858,0.5738,0.5114,0.5196,0.4976,0.4986,0.4752
A03,0.7808,0.7807,1.0000,0.7488,0.7348,0.8481,0.8101,0.7181,0.7162,0.7141,...,0.6193,0.5914,0.6499,0.6216,0.6201,0.5252,0.6123,0.6131,0.6108,0.5523
A04,0.7677,0.8987,0.7488,1.0000,0.8766,0.7642,0.8775,0.9298,0.8976,0.7910,...,0.5635,0.5647,0.5862,0.4959,0.6634,0.5506,0.5771,0.5436,0.5822,0.5037
A05,0.7582,0.8605,0.7348,0.8766,1.0000,0.7277,0.8159,0.8050,0.8306,0.7891,...,0.6396,0.6042,0.5617,0.4860,0.5999,0.4987,0.5077,0.5138,0.5346,0.5740


qwen (2032, 2032)


,A00,A01,A02,A03,A04,A05,A06,A07,A08,A09,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A01,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A02,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A03,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A04,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [7]:
intersection = sorted(list(intersection))

In [8]:
len(intersection)

1696

In [9]:
for k, v in matrices.items():
    matrices[k] = v.loc[intersection, intersection]

In [10]:
for k, v in matrices.items():
    print(k)
    display(v.head(3))

statistic


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.0000,0.0400,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A02,0.0400,1.0000,0.0000,0.0035,0.0000,0.0000,0.0000,0.0043,0.0020,0.0042,...,0.0014,0.0008,0.0015,0.0010,0.0027,0.0008,0.0010,0.0016,0.0009,0.0004
A03,0.0000,0.0000,1.0000,0.0008,0.0000,0.0000,0.0133,0.0003,0.0021,0.0000,...,0.0001,0.0001,0.0002,0.0000,0.0000,0.0000,0.0001,0.0002,0.0000,0.0001


bert


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.0000,0.8591,0.6663,0.8590,0.8232,0.7387,0.8297,0.8829,0.8726,0.8202,...,0.8054,0.7709,0.7201,0.7007,0.7522,0.7798,0.7637,0.7390,0.7087,0.7486
A02,0.8591,1.0000,0.7781,0.9732,0.8877,0.8647,0.9421,0.9341,0.9137,0.8221,...,0.8340,0.8226,0.8044,0.8205,0.8043,0.7879,0.8504,0.8559,0.8342,0.8135
A03,0.6663,0.7781,1.0000,0.7914,0.7905,0.9191,0.8056,0.7389,0.7291,0.6688,...,0.7761,0.7331,0.7604,0.8092,0.7333,0.6571,0.7582,0.7807,0.8348,0.7784


medbert


...

,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.0000,0.7000,0.0000,0.0000,0.0000,0.0000,0.0000,0.6000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A02,0.7000,1.0000,0.7000,0.6000,0.0000,0.0000,0.0000,0.8000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A03,0.0000,0.7000,1.0000,0.7000,0.6000,0.7000,0.0000,0.8000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


doc


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,1.0000,0.7428,0.7808,0.7677,0.7582,0.7838,0.7795,0.7592,0.7595,0.7714,...,0.6238,0.6441,0.6317,0.5498,0.6119,0.5632,0.5865,0.6073,0.6510,0.5643
A02,0.7428,1.0000,0.7807,0.8987,0.8605,0.7332,0.8561,0.8724,0.8412,0.7605,...,0.5522,0.5543,0.5303,0.4858,0.5738,0.5114,0.5196,0.4976,0.4986,0.4752
A03,0.7808,0.7807,1.0000,0.7488,0.7348,0.8481,0.8101,0.7181,0.7162,0.7141,...,0.6193,0.5914,0.6499,0.6216,0.6201,0.5252,0.6123,0.6131,0.6108,0.5523


qwen


,A01,A02,A03,A04,A05,A06,A07,A08,A09,A15,...,Z90,Z91,Z92,Z93,Z94,Z95,Z96,Z97,Z98,Z99
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
A01,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.5000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A02,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A03,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [33]:
pearsonr = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)

In [34]:
# for method1, method2 in tqdm(itertools.combinations(matrices.keys(), 2)):
for llm, baseline in tqdm(itertools.product(LLMS, BASELINES)):
    print("Computing estimate for {} and {}".format(llm, baseline))
    pearsonr.loc[llm, baseline] = sts.pearsonr(matrices[llm].values.flatten(), matrices[baseline].values.flatten()).statistic

2it [00:00, 17.60it/s]

Computing estimate for deepseek and statistic
Computing estimate for deepseek and medbert
Computing estimate for deepseek and bert
Computing estimate for deepseek and doc


6it [00:00, 17.82it/s]

Computing estimate for qwen and statistic
Computing estimate for qwen and medbert
Computing estimate for qwen and bert
Computing estimate for qwen and doc


10it [00:00, 15.96it/s]

Computing estimate for yandex and statistic
Computing estimate for yandex and medbert
Computing estimate for yandex and bert


14it [00:00, 16.26it/s]

Computing estimate for yandex and doc
Computing estimate for mlm and statistic
Computing estimate for mlm and medbert
Computing estimate for mlm and bert


16it [00:00, 16.76it/s]

Computing estimate for mlm and doc


In [35]:
pearsonr

,statistic,medbert,bert,doc
deepseek,0.0422,0.0412,-0.0231,0.1729
qwen,0.0465,0.0512,0.0327,0.1270
yandex,0.5158,0.1183,0.0701,0.1819
mlm,0.3449,0.0382,0.0239,0.0532


In [11]:
spearmanr = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)
spearmanp = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)

In [12]:
# for method1, method2 in tqdm(itertools.combinations(matrices.keys(), 2)):
for llm, baseline in tqdm(itertools.product(LLMS, BASELINES)):
    print("Computing estimate for {} and {}".format(llm, baseline))
    statistic, p_value = sts.spearmanr(matrices[llm].values.flatten(), matrices[baseline].values.flatten())
    spearmanr.loc[llm, baseline] = statistic
    spearmanp.loc[llm, baseline] = p_value
    spearmanr.to_csv("spearman.csv")
    spearmanp.to_csv("spearman_p_value.csv")

0it [00:00, ?it/s]

Computing estimate for deepseek and statistic


1it [00:00,  2.28it/s]

Computing estimate for deepseek and medbert


2it [00:00,  2.41it/s]

...

14it [00:05,  2.36it/s]

Computing estimate for mlm and bert


15it [00:06,  2.26it/s]

Computing estimate for mlm and doc


16it [00:06,  2.38it/s]


In [13]:
spearmanr

,statistic,medbert,bert,doc
deepseek,0.0064,0.0244,-0.0217,0.1390
qwen,0.0714,0.0291,0.0292,0.0929
yandex,0.0698,0.0701,0.0565,0.0925
mlm,-0.0017,0.0051,0.0027,0.0076


In [14]:
spearmanp

,statistic,medbert,bert,doc
deepseek,0.0000,0.0000,0.0000,0.0000
qwen,0.0000,0.0000,0.0000,0.0000
yandex,0.0000,0.0000,0.0000,0.0000
mlm,0.0043,0.0000,0.0000,0.0000


In [22]:
def compute_bootstrapped_score(
        y_test: np.array, 
        y_prob: np.array, 
        scorer: Callable[[np.array, np.array], Any], 
        m_sample: int = None, 
        stratum_vals: np.array =None
    ):
    assert not isinstance(y_test, pd.Series), "y_test should be np.array"
    assert not isinstance(y_prob, pd.Series), "y_prob should be np.array"

    idx = np.array(range(len(y_test)))
    if m_sample is None: m_sample = len(y_test) #bootstrap sample size
        
    if stratum_vals is not None: #select equal number of samples from each category
        idx_bs = [] 
        for val in set(stratum_vals):
            stratum_idx = idx[stratum_vals == val] 
            idx_bs += np.random.choice(stratum_idx, size=len(stratum_idx), replace=True).tolist()
    else:
        idx_bs = np.random.choice(idx, size=m_sample, replace=True)

    try:
        return scorer(y_test[idx_bs], y_prob[idx_bs]).statistic
    except Exception as e:
        print("WARNING: Bootstrapping failed for", scorer.func.__name__ if isinstance(scorer, partial) else scorer.__name__, "with error", e)
        return np.nan
    
def compute_ci(
    y_test: np.array, 
    y_prob: np.array, 
    stratum_vals: np.array = None, 
    n_bootstraps: int = 1000, 
    m_sample: int = None, 
    scorer: Callable[[np.array, np.array], Any] = sts.spearmanr,
    alpha: float = 0.05, #95% CI
    verbose: int = 1, 
    return_se: bool = False
):
    assert len(y_test) == len(y_prob), "y_test and y_prob should have the same lengths"
        
    scores = []
    if verbose > 0:
        print(f"Bootstrap scores computing for {scorer.func.__name__ if isinstance(scorer, partial) else scorer.__name__}...")
        for _ in tqdm(range(n_bootstraps)):
            scores.append(compute_bootstrapped_score(y_test, y_prob, scorer, stratum_vals=stratum_vals, m_sample=m_sample))
    else:
        for _ in range(n_bootstraps):
            scores.append(compute_bootstrapped_score(y_test, y_prob, scorer, stratum_vals=stratum_vals, m_sample=m_sample))
    scores = np.array(scores)
    
    nans_share = np.sum(np.isnan(scores).astype(int))/len(scores)
    if nans_share > 0.5: #empirical threshold, you can change it if you have better solution
        print(f"WARNING: There is {nans_share*100:.0f}% NaNs in bootstrapped scores for {scorer.func.__name__ if isinstance(scorer, partial) else scorer.__name__}")
        random_idxs = np.random.choice(list(range(len(y_test))), size=20, replace=False)
        print("       random 20 entries from y_test:", y_test[random_idxs])
        print("corresponging 20 entries from y_prob:", y_prob[random_idxs])
        if return_se:
            return np.nan, np.nan, np.nan
        else:
            return np.nan, np.nan
    
    estimation = np.nanmean(scores)
    se = np.nanstd(scores)
    perc = sts.norm.ppf(1 - alpha/2)
    e_perc = se * perc
    
    if verbose > 1:
        plt.figure(figsize=(4, 2.5))
        plt.hist(scores, bins=50)
        plt.axvline(x = estimation, color = 'tab:orange', label = 'mean')
        plt.axvline(x = estimation - e_perc, color = 'tab:red', label = f'mean - e_{1-alpha:.2f}')
        plt.axvline(x = estimation + e_perc, color = 'tab:red', label = f'mean + e_{1-alpha:.2f}')
        plt.show()
    
    if return_se:
        return estimation, e_perc, se
    else:
        return estimation, e_perc

In [23]:
spearman_mean = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)
spearman_se = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)
spearman_e_perc = pd.DataFrame(data=-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)

In [24]:
from multiprocessing import Pool

def _worker_pair(args):
    """Compute CI for a single (llm, baseline) pair. Top-level for pickling."""
    (llm, baseline, x_flat, y_flat, kwargs) = args
    # call your existing function (single-process bootstrap inside)
    r, e_perc, se = compute_ci(
        x_flat, y_flat,
        return_se=True,
        **kwargs
    )
    return llm, baseline, r, e_perc, se

if __name__ == "__main__":  # important for Windows / notebooks
    # prepare outputs
    spearman_mean   = pd.DataFrame(-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)
    spearman_se     = pd.DataFrame(-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)
    spearman_e_perc = pd.DataFrame(-np.ones((len(LLMS), len(BASELINES))), columns=BASELINES, index=LLMS)

    # build tasks (pre-flatten to avoid doing it in workers repeatedly)
    common_kwargs = dict(
        scorer=sts.spearmanr,
        n_bootstraps=1000,   # adjust as you like
        m_sample=None,
        stratum_vals=None,
        alpha=0.05,
        verbose=1            # silence per-worker prints
    )
    tasks = []
    for llm, baseline in itertools.product(LLMS, BASELINES):
        x_flat = matrices[llm].values.ravel()
        y_flat = matrices[baseline].values.ravel()
        tasks.append((llm, baseline, x_flat, y_flat, common_kwargs))

    # run in parallel across pairs
    n_jobs = None  # or an int like 8
    with Pool(processes=n_jobs) as pool:
        for llm, baseline, r, e_perc, se in tqdm(pool.imap_unordered(_worker_pair, tasks, chunksize=1)):
            spearman_mean.loc[llm, baseline]   = r
            spearman_se.loc[llm, baseline]     = se
            spearman_e_perc.loc[llm, baseline] = e_perc
    
    spearman_mean.to_csv("spearman_mean.csv")
    spearman_se.to_csv("spearman_se.csv")
    spearman_e_perc.to_csv("spearman_e_perc.csv")

16it [16:26, 61.63s/it]


In [25]:
spearman_mean

,statistic,medbert,bert,doc
deepseek,0.0064,0.0244,-0.0217,0.1390
qwen,0.0714,0.0291,0.0293,0.0929
yandex,0.0698,0.0701,0.0565,0.0926
mlm,-0.0017,0.0052,0.0027,0.0076


In [32]:
spearman_e_perc

,statistic,medbert,bert,doc
deepseek,0.0012,0.0012,0.0011,0.0012
qwen,0.0013,0.0013,0.0012,0.0012
yandex,0.0014,0.0012,0.0012,0.0011
mlm,0.0012,0.0012,0.0012,0.0012


In [26]:
spearman_se

,statistic,medbert,bert,doc
deepseek,0.0006,0.0006,0.0006,0.0006
qwen,0.0007,0.0007,0.0006,0.0006
yandex,0.0007,0.0006,0.0006,0.0006
mlm,0.0006,0.0006,0.0006,0.0006


In [3]:
spearman_mean = pd.read_csv('spearman_mean.csv')
spearman_se = pd.read_csv('spearman_se.csv')